In [ ]:
from epx import FREDJob, FREDModelConfig, SynthPop
import time
import pandas as pd

import os
from pathlib import Path
os.environ["FRED_PROJECT"] = str(Path.cwd())

# Ground logistics model

This model represents a ground transportation network. Cities represent nodes in the network and roads are edges. A table of trucks agents is read in at the beginning of the simulation. There are no human agents in the model. 

## Model components

- `city_locations.fred`: contains the GENERATE\_LOCATIONS condition that creates a network of cities
- `trucks.fred`: contains the GENERATE\_TRUCKS condition that reads the trucks agent file and the MOVE\_TRUCKS condition that picks next locations for trucks to travel to
- `travel_disruption.fred`: contains the DISRUPTION condition that creates travel disruptions at certain nodes as defined by an end user

In [ ]:
ground_shipping_logistics_config = FREDModelConfig(
             synth_pop=SynthPop("US_2010.v5", ["None"]),
             start_date="2020-01-01",
             end_date="2020-02-01",
         )

fred_ref_files = {
    "data/routes.csv":"data/routes.csv",
    "data/city_locations.txt":"data/city_locations.txt",
    "data/truck_capacity.txt":"data/truck_capacity.txt",
    "data/trucks.txt":"data/trucks.txt"}

fred_include_files = [
    "model/main.fred",
    "model/city_locations.fred",
    "model/trucks.fred",
    "model/travel_disruption.fred"]

ground_shipping_logistics_job = FREDJob(
    config=[ground_shipping_logistics_config],
    tags=["cl_ground_shipping_logistics","epx-client-1.2.0"],
    fred_version="11.0.1",
    fred_files=fred_include_files,
    ref_files=fred_ref_files
)

ground_shipping_logistics_job.execute(300)

str(ground_shipping_logistics_job.status)

## Data outputs

Let's take a look at the output `travel_info.csv`. The data summarizes the progress of each truck on the journey from its most recently visited city to its next destination by recording the remaining distance left to travel on each day of the simulation.

In [ ]:
travel_log_df = ground_shipping_logistics_job.results.csv_output('travel_log.csv')
travel_log_df

In [ ]:
ground_shipping_logistics_job.delete(interactive=False)